In [ ]:
#!pip install surprise

In [ ]:
#!pip install konlpy

In [ ]:
#!pip install transformers

In [ ]:
#!pip install sentence_transformers

In [ ]:
#!pip install pyspark

In [ ]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
from sklearn.model_selection import train_test_split
# from surprise import KNNBasic
# from surprise import accuracy
# from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import jaccard_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import nltk
from nltk.stem import WordNetLemmatizer
# from konlpy.tag import Okt
# from konlpy.tag import Mecab
# from gensim.models import Word2Vec
# from transformers import BertTokenizer, BertModel
# from sentence_transformers import SentenceTransformer
# from pyspark.ml.recommendation import ALS
# from pyspark.sql import SparkSession
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

import torch
from collections import Counter

In [ ]:
drive.mount('./content/')

Mounted at ./content/


In [ ]:
folder_dir = '/content/content/MyDrive/KMU/'
apply_train = pd.read_csv(folder_dir + 'apply_train.csv')
company = pd.read_csv(folder_dir + 'company.csv')
recruitment = pd.read_csv(folder_dir + 'recruitment.csv')
resume = pd.read_csv(folder_dir + 'resume.csv')
resume_certificate = pd.read_csv(folder_dir + 'resume_certificate.csv')
resume_education = pd.read_csv(folder_dir + 'resume_education.csv')
resume_language = pd.read_csv(folder_dir + 'resume_language.csv')
sample_submission = pd.read_csv(folder_dir + 'sample_submission.csv')

In [ ]:
apply_train

,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112
...,...,...
57941,U02270,R03430
57942,U02640,R04987
57943,U08238,R01342
57944,U01296,R06363


이력서 관련 데이터

In [ ]:
resume.head()

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스)
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,24,섬유;봉제;가방;의류
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,0,NaN


In [ ]:
resume_certificate.head()

,resume_seq,certificate_contents
0,U06421,손해보험사
1,U01278,2종운전면허증
2,U01008,자동차운전면허 2종 보통
3,U03248,운전면허2종보통
4,U07483,메이크업3급


In [ ]:
resume_certificate

,resume_seq,certificate_contents
0,U06421,손해보험사
1,U01278,2종운전면허증
2,U01008,자동차운전면허 2종 보통
3,U03248,운전면허2종보통
4,U07483,메이크업3급
...,...,...
12970,U07453,컬러리스트산업기사
12971,U06358,컴퓨터그래픽스운용기능사
12972,U03248,수치제어밀링기능사
12973,U08157,주산2급


In [ ]:
resume_certificate

,resume_seq,certificate_contents
0,U06421,손해보험사
1,U01278,2종운전면허증
2,U01008,자동차운전면허 2종 보통
3,U03248,운전면허2종보통
4,U07483,메이크업3급
...,...,...
12970,U07453,컬러리스트산업기사
12971,U06358,컴퓨터그래픽스운용기능사
12972,U03248,수치제어밀링기능사
12973,U08157,주산2급


In [ ]:
resume_education.head()

,resume_seq,hischool_type_seq,hischool_special_type,hischool_nation,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U01419,21,일반고,사립,남자학교,3,5,5,0,3,NaN,NaN,9,60.0
1,U03375,21,일반고,사립,여자학교,3,5,5,0,3,NaN,NaN,4,80.0
2,U06523,21,일반고,사립,남여공학,3,5,5,0,3,NaN,NaN,8,70.0
3,U06619,21,일반고,사립,남여공학,5,5,5,0,5,NaN,NaN,8,80.0
4,U05015,16,특성화고,공립,남여공학,3,5,5,0,3,NaN,NaN,9,80.0


In [ ]:
resume_language.head()

,resume_seq,language,exam_name,score
0,U01774,2,4,742.42
1,U04892,2,4,888.89
2,U01859,2,4,500.00
3,U02475,2,4,500.00
4,U03776,4,5,595.96


채용 공고 관련 데이터

In [ ]:
recruitment.head()

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
0,R02264,3.0,NaN,NaN,0,0,2507;2707;2810,4,8,1,NaN
1,R06317,3.0,NaN,NaN,0,0,2204;2205;2707,3,2,1,NaN
2,R04017,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,1,NaN
3,R02865,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,1,NaN
4,R04890,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN


In [ ]:
company.head()

,recruitment_seq,company_type_seq,supply_kind,employee
0,R02073,2,514,20
1,R03274,2,402,90
2,R02195,2,514,20
3,R03372,4,100,60
4,R00867,2,402,590


In [ ]:
resume_certificate['certificate_contents_count'] = resume_certificate.groupby('resume_seq')['certificate_contents'].transform('count')

In [ ]:
resume_certificate

,resume_seq,certificate_contents,certificate_contents_count
0,U06421,손해보험사,6
1,U01278,2종운전면허증,4
2,U01008,자동차운전면허 2종 보통,1
3,U03248,운전면허2종보통,3
4,U07483,메이크업3급,9
...,...,...,...
12970,U07453,컬러리스트산업기사,3
12971,U06358,컴퓨터그래픽스운용기능사,2
12972,U03248,수치제어밀링기능사,3
12973,U08157,주산2급,4


In [ ]:
# 이력서 관련 데이터 병합
resume = pd.merge(resume, resume_certificate, how = 'left', on = 'resume_seq')
resume = pd.merge(resume, resume_education, how = 'left', on = 'resume_seq')
resume = pd.merge(resume, resume_language, how = 'left', on = 'resume_seq')

# 채용 공고 관련 데이터 병합
recruitment = pd.merge(recruitment, company, how = 'left', on = 'recruitment_seq')

In [ ]:
resume['degree_rating'] = resume['degree'] - 1

In [ ]:
resume['degree_rating'].value_counts()

3    13827
2      761
4      640
1      357
5        9
Name: degree_rating, dtype: int64

In [ ]:
resume['univ_score_rating'] = pd.cut(resume['univ_score'], bins = [0, 60, 80, 100], labels = [1, 2, 3])

In [ ]:
resume['univ_score_rating'].value_counts()

2    11359
3     2229
1     1465
Name: univ_score_rating, dtype: int64

In [ ]:
# resume['career_month_rating'] = pd.cut(resume['career_month']//12, bins = [0, 1, 5, 10, np.inf], labels = [1, 2, 3, 4])

In [ ]:
# resume['career_month_rating'].value_counts()

In [ ]:
resume['score_rating'] = pd.cut(resume['score'], bins = [0, 600, 700, 800, 900, 1000], labels = [1, 2, 3, 4, 5])

In [ ]:
resume['score_rating'].value_counts()

1    619
4    424
3    350
5    233
2    210
Name: score_rating, dtype: int64

In [ ]:
recruitment['education'].value_counts() # 요구 학위 = degree_rating

3    2862
4    2188
2    1644
6       1
Name: education, dtype: int64

In [ ]:
resume['certificate_contents_count'] = resume['certificate_contents_count'].fillna(0.0)

In [ ]:
resume['certificate_contents_count'].value_counts()

2.0     3007
3.0     2805
1.0     2612
0.0     2516
4.0     2032
5.0     1230
6.0      792
7.0      344
8.0      145
9.0       91
10.0      20
Name: certificate_contents_count, dtype: int64

In [ ]:
resume['certificate_contents_rating'] = pd.cut(resume['certificate_contents_count'], bins = [-1, 0, 5, 10], labels = [1, 2, 3])

In [ ]:
resume['certificate_contents_rating'].value_counts()

2    11686
1     2516
3     1392
Name: certificate_contents_rating, dtype: int64

In [ ]:
recruitment['qualifications'].value_counts()

1    4492
2    2164
0      39
Name: qualifications, dtype: int64

In [ ]:
rating_info = resume.columns[resume.columns.str.contains('rating')]

In [ ]:
resume[rating_info] = resume[rating_info].fillna(1)
resume[rating_info] = resume[rating_info].astype(int)

In [ ]:
resume['rating_total'] = resume[rating_info].sum(axis = 1)

In [ ]:
resume = resume.sort_values(by = 'resume_seq', ascending = True)
resume = resume.drop_duplicates(['resume_seq'], keep = 'first')
apply_train = pd.merge(apply_train, resume[['resume_seq', 'rating_total']], left_on = 'resume_seq', right_on = 'resume_seq')

# 콘텐츠 기반

In [ ]:
# '2종'이 포함된 데이터에 대해서만 '2종' 남기기
resume.certificate_contents = [text if '2종' not in str(text) else '2종' for text in resume.certificate_contents]
# '1종'이 포함된 데이터에 대해서만 '1종' 남기기
resume.certificate_contents = [text if '1종' not in str(text) else '1종' for text in resume.certificate_contents]
# 'MCAS MASTER'
resume.certificate_contents = [text if 'MCAS MASTER' not in str(text) else 'MCAS MASTER' for text in resume.certificate_contents]

resume['text_keyword'] = resume['text_keyword'].str.replace(';', '')
recruitment['text_keyword'] = recruitment['text_keyword'].str.replace(';', '')

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
resume_tf_vec = tfidf_vectorizer.fit_transform(resume['text_keyword'].astype(str))
recruitment_tf_vec = tfidf_vectorizer.transform(recruitment['text_keyword'].astype(str))

# merge_resume = pd.DataFrame(resume_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# merge_recruitment = pd.DataFrame(recruitment_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# merge_resume = pd.concat([pd.DataFrame(resume_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names_out()), resume['degree']], axis = 1)
# merge_recruitment = pd.concat([pd.DataFrame(recruitment_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names_out()), recruitment['education']], axis = 1)

# 코사인 유사도 계산
similarity_matrix1 = cosine_similarity(resume_tf_vec, recruitment_tf_vec)
#similarity_matrix1 = linear_kernel(merge_resume, merge_recruitment)

# similarity_matrix2 = cosine_similarity(resume['degree_rating'].values.reshape(-1, 1), recruitment['education'].values.reshape(-1, 1))
# #similarity_matrix2 = linear_kernel(merge_resume, merge_recruitment)

# similarity_matrix3 = cosine_similarity(resume['certificate_contents_rating'].values.reshape(-1, 1), recruitment['qualifications'].values.reshape(-1, 1))

similarity_matrix = similarity_matrix1

user_item_matrix = apply_train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value = 0)
user_item_matrix[user_item_matrix > 1] = 1

similarity_matrix_df = pd.DataFrame(similarity_matrix, index = resume.resume_seq, columns = recruitment.recruitment_seq)

In [ ]:
# user_list = list(similarity_matrix_df.index)
# recommended = []

# for resume_seq in user_list:
#     # 해당 사용자가 지원한 채용 공고
#     applied_jobs = set(user_item_matrix.loc[resume_seq][user_item_matrix.loc[resume_seq] == 1].index)

#     similarity_recruitment = list(enumerate(similarity_matrix_df.loc[resume_seq]))
#     similarity_recruitment = sorted(similarity_recruitment, key=lambda x: x[1], reverse=True)

#     top_N = 5  # 상위 5개 추천된 채용 공고
#     recommended_seqs = [recruitment['recruitment_seq'][recommend[0]] for recommend in similarity_recruitment[1: top_N*2+1]]

#     # 이미 지원한 채용 공고를 제외하고 상위 N개를 추천
#     filtered_recommended = [seq for seq in recommended_seqs if seq not in applied_jobs][:top_N]

#     for recommended_seq in filtered_recommended:
#         recommended.append([resume_seq, recommended_seq])

In [ ]:
# recommended_predictions = pd.DataFrame(recommended, columns=['resume_seq', 'recruitment_seq'])
# recommended_predictions = recommended_predictions.sort_values(by = 'resume_seq', ascending=True)
# recommended_predictions.to_csv('contents_baseline.csv', index = False)

# 협업 필터링

In [ ]:
def recall5(answer_df, submission_df):
    """
    Calculate recall@5 for given dataframes.

    Parameters:
    - answer_df: DataFrame containing the ground truth
    - submission_df: DataFrame containing the predictions

    Returns:
    - recall: Recall@5 value
    """

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # Check if each primary_col entry has exactly 5 secondary_col predictions
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # Check for NULL values in the predicted secondary_col
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # Check for duplicates in the predicted secondary_col for each primary_col
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)

    return recall

In [ ]:
rating_apply_train = apply_train.pivot_table('rating_total', index = 'resume_seq', columns = 'recruitment_seq')
rating_apply_train = rating_apply_train.fillna(0)

In [ ]:
#recall5(val,val_prediction)

In [ ]:
from tqdm.auto import tqdm
import gc

In [ ]:
# 자카드 유사도
def jaccard_similarity(user1, user2):
  intersection = np.logical_and(user1, user2)
  union = np.logical_or(user1, user2)
  return intersection.sum() / union.sum()

In [ ]:
def adjusted_cosine_similarity(matrix):
  centered_matrix = matrix - matrix.mean(axis=1)
  similarity = np.dot(centered_matrix, centered_matrix.T) / (np.linalg.norm(centered_matrix, axis=1, keepdims=True) @ np.linalg.norm(centered_matrix, axis=1, keepdims=True).T)
  return similarity

In [ ]:
train, val = [], []
apply_train_groupby = apply_train.groupby('resume_seq')['recruitment_seq'].apply(list)

for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
  random.shuffle(iids)
  for iid in iids[:-1]:
    train.append([uid,iid])
  val.append([uid, iids[-1]])

train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
val = pd.DataFrame(val, columns=['resume_seq', 'recruitment_seq'])
pred = apply_train.copy()

train_user_item_matrix = train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
pred_user_item_matrix = pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)

del train
del pred
gc.collect()

0

In [ ]:
U, Sigma, Vt = np.linalg.svd(train_user_item_matrix, full_matrices=False)

k = 50
U_k = U[:, :k]
Sigma_k = np.diag(Sigma[:k])
Vt_k = Vt[:k, :]

train_user_predicted_ratings_svd = np.dot(U_k, np.dot(Sigma_k, Vt_k))

del k
del U_k
del Sigma_k
del Vt_k
gc.collect()

U, Sigma, Vt = np.linalg.svd(pred_user_item_matrix, full_matrices=False)

k = 50
U_k = U[:, :k]
Sigma_k = np.diag(Sigma[:k])
Vt_k = Vt[:k, :]

pred_user_predicted_ratings_svd = np.dot(U_k, np.dot(Sigma_k, Vt_k))

del k
del U_k
del Sigma_k
del Vt_k
gc.collect()

0

In [ ]:
U, Sigma, Vt = np.linalg.svd(train_user_item_matrix.T, full_matrices=False)

k = 50
U_k = U[:, :k]
Sigma_k = np.diag(Sigma[:k])
Vt_k = Vt[:k, :]

train_item_predicted_ratings_svd = np.dot(U_k, np.dot(Sigma_k, Vt_k))

del k
del U_k
del Sigma_k
del Vt_k
gc.collect()

U, Sigma, Vt = np.linalg.svd(pred_user_item_matrix.T, full_matrices=False)

k = 50
U_k = U[:, :k]
Sigma_k = np.diag(Sigma[:k])
Vt_k = Vt[:k, :]

pred_item_predicted_ratings_svd = np.dot(U_k, np.dot(Sigma_k, Vt_k))

del k
del U_k
del Sigma_k
del Vt_k
gc.collect()

0

In [ ]:
# 코사인 유사도
train_user_similarity = cosine_similarity(train_user_item_matrix)
train_item_similarity = cosine_similarity(train_user_item_matrix.T)

pred_user_similarity = cosine_similarity(pred_user_item_matrix)
pred_item_similarity = cosine_similarity(pred_user_item_matrix.T)

train_user_predicted_scores_cosine = train_user_similarity.dot(train_user_item_matrix)
train_item_predicted_scores_cosine = train_user_item_matrix.dot(train_item_similarity)

pred_user_predicted_scores_cosine = pred_user_similarity.dot(pred_user_item_matrix)
pred_item_predicted_scores_cosine = pred_user_item_matrix.dot(pred_item_similarity)

del train_user_similarity
del train_item_similarity
del pred_user_similarity
del pred_item_similarity

In [ ]:
# 피어슨 상관계수
train_user_pearson = np.corrcoef(train_user_item_matrix)
train_item_pearson = np.corrcoef(train_user_item_matrix.T)

pred_user_pearson = np.corrcoef(pred_user_item_matrix)
pred_item_pearson = np.corrcoef(pred_user_item_matrix.T)

train_user_predicted_scores_pearson = train_user_pearson.dot(train_user_item_matrix)
train_item_predicted_scores_pearson = train_user_item_matrix.dot(train_item_pearson)

pred_user_predicted_scores_pearson = pred_user_pearson.dot(pred_user_item_matrix)
pred_item_predicted_scores_pearson = pred_user_item_matrix.dot(pred_item_pearson)

del train_user_pearson
del train_item_pearson
del pred_user_pearson
del pred_item_pearson

In [ ]:
train_user_predicted_ratings_svd = pd.DataFrame(train_user_predicted_ratings_svd, index = train_item_predicted_scores_pearson.index, columns = train_item_predicted_scores_pearson.columns)
pred_user_predicted_ratings_svd = pd.DataFrame(pred_user_predicted_ratings_svd, index = pred_item_predicted_scores_pearson.index, columns = pred_item_predicted_scores_pearson.columns)

In [ ]:
pred_item_predicted_ratings_svd.shape

(6695, 8482)

In [ ]:
a = train_user_item_matrix.columns
b = set(similarity_matrix_df).difference(set(a))

similarity_matrix_df_train = similarity_matrix_df.drop(list(b), axis = 1)

job1_alpha = 0.65
job2_alpha = 0.35
contents_alpha = 0.1
train_recommendation = []
for idx, user in tqdm(enumerate(train_user_item_matrix.index)):
  applied_jobs = set(train_user_item_matrix.loc[user][train_user_item_matrix.loc[user] == 1].index)

  #sorted_content_indices = np.array(similarity_matrix_df_train.loc[user].values)[0].argsort()[::-1]
  #recommended_content = [job for job in train_user_item_matrix.columns[sorted_content_indices] if job not in applied_jobs][:15]

  # 매칭 기반 추천 점수 (높은 점수부터 정렬)
  rating_scores = rating_apply_train.loc[user].values
  sorted_rating_indices = rating_scores.argsort()[::-1]
  recommened_rating = rating_apply_train.columns[sorted_rating_indices]

  recommended_contents_scores = similarity_matrix_df_train.loc[user].values #* similarity_matrix_df_train.T.loc[idx].values
  sorted_contents_indices = recommended_contents_scores.argsort()[::-1]
  recommended_contents = [job for job in train_user_item_matrix.columns[sorted_contents_indices] if job not in applied_jobs]
  #recommended_contents = [item for item in recommened_rating if item in recommended_contents]

  # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
  combined_scores1 = (train_item_predicted_scores_cosine.loc[user].values *
                  train_user_predicted_scores_cosine[idx])

  sorted_job_indices1 = combined_scores1.argsort()[::-1]
  recommended_jobs1 = [job for job in train_user_item_matrix.columns[sorted_job_indices1] if job not in applied_jobs][:10]
  #recommended_jobs1 = [item for item in recommened_rating if item in recommended_jobs1]

  combined_scores2 = ((train_item_predicted_scores_pearson.loc[user].values) *
                  (train_user_predicted_scores_pearson[idx]))

  sorted_job_indices2 = combined_scores2.argsort()[::-1]
  recommended_jobs2 = [job for job in train_user_item_matrix.columns[sorted_job_indices2] if job not in applied_jobs][:10]
  #recommended_jobs2 = [item for item in recommened_rating if item in recommended_jobs2]

  combined_scores3 = (train_user_predicted_ratings_svd.loc[user].values *
                    pd.DataFrame(train_item_predicted_ratings_svd)[idx])
  #                 #np.array(similarity_matrix_df_train.loc[user].values)[0]

  sorted_job_indices3 = combined_scores3.argsort()[::-1]
  recommended_jobs3 = [job for job in train_user_item_matrix.columns[sorted_job_indices3] if job not in applied_jobs][:5]
  #recommended_jobs3 = [item for item in recommened_rating if item in recommended_jobs3]

  # 두 목록 병합
  # combined_ensemble_scores = combined_scores1 * job1_alpha + combined_scores2 * job2_alpha
  # sorted_ensemble_indices2 = combined_ensemble_scores.argsort()[::-1]
  # recommended_ensemble = [job for job in train_user_item_matrix.columns[sorted_ensemble_indices2] if job not in applied_jobs][:10]

  combined_recommendations = recommended_jobs1 + recommended_jobs2 + recommended_jobs3 #+ recommended_contents

  # 목록에서 중복 항목을 제거하고 빈도를 계산
  recommendation_counter = Counter(combined_recommendations)

  # 빈도수를 기반으로 상위 5개 추천 얻기
  final_recommendations = [item for item, _ in recommendation_counter.most_common(5)]
  final_recommendations = [item for item in recommened_rating if item in final_recommendations]
  #final_recommendations = [item for item in recommended_contents if item in final_recommendations]

  for job in final_recommendations:
    train_recommendation.append([user, job])

val_prediction = pd.DataFrame(train_recommendation, columns=['resume_seq', 'recruitment_seq'])

print(recall5(val, val_prediction))

del val
gc.collect()

a = pred_user_item_matrix.columns
b = set(similarity_matrix_df).difference(set(a))

similarity_matrix_df_pred = similarity_matrix_df.drop(list(b), axis = 1)

pred_recommendation = []
for idx, user in tqdm(enumerate(pred_user_item_matrix.index)):
  applied_jobs = set(pred_user_item_matrix.loc[user][pred_user_item_matrix.loc[user] == 1].index)

  #sorted_content_indices = np.array(similarity_matrix_df_pred.loc[user].values)[0].argsort()[::-1]
  #recommended_content = [job for job in pred_user_item_matrix.columns[sorted_content_indices] if job not in applied_jobs][:15]

  # similarity_matrix_df.drop(list(b), axis = 1).loc[user].values

  # 매칭 기반 추천 점수 (높은 점수부터 정렬)
  rating_scores = rating_apply_train.loc[user].values
  sorted_rating_indices = rating_scores.argsort()[::-1]
  recommened_rating = rating_apply_train.columns[sorted_rating_indices]

  #해당 사용자의 추천 점수 (높은 점수부터 정렬)
  combined_scores1 = (pred_item_predicted_scores_cosine.loc[user].values *
                    pred_user_predicted_scores_cosine[idx])

  sorted_job_indices1 = combined_scores1.argsort()[::-1]
  recommended_jobs1 = [job for job in pred_user_item_matrix.columns[sorted_job_indices1] if job not in applied_jobs][:10]
  #recommended_jobs1 = [item for item in recommened_rating if item in recommended_jobs1]

  combined_scores2 = ((pred_item_predicted_scores_pearson.loc[user].values) *
                    (pred_user_predicted_scores_pearson[idx]))

  sorted_job_indices2 = combined_scores2.argsort()[::-1]
  recommended_jobs2 = [job for job in pred_user_item_matrix.columns[sorted_job_indices2] if job not in applied_jobs][:10]
  #recommended_jobs2 = [item for item in recommened_rating if item in recommended_jobs2]

  combined_scores3 = (pred_user_predicted_ratings_svd.loc[user].values *
                    pd.DataFrame(pred_item_predicted_ratings_svd)[idx])
  #                   #np.array(similarity_matrix_df_pred.loc[user].values)[0]

  sorted_job_indices3 = combined_scores3.argsort()[::-1]
  recommended_jobs3 = [job for job in pred_user_item_matrix.columns[sorted_job_indices3] if job not in applied_jobs][:10]
  # #recommended_jobs3 = [item for item in recommened_rating if item in recommended_jobs3]

  # recommended_contents_scores = similarity_matrix_df_pred.loc[user].values #* similarity_matrix_df_pred.T.loc[idx].values
  # sorted_contents_indices = recommended_contents_scores.argsort()[::-1]
  # recommended_contents = [job for job in pred_user_item_matrix.columns[sorted_contents_indices] if job not in applied_jobs][:10]
  # recommended_contents = [item for item in recommened_rating if item in recommended_contents]

  # 두 목록 병합
  # combined_ensemble_scores = combined_scores1 * job1_alpha + combined_scores2 * job2_alpha
  # sorted_ensemble_indices2 = combined_ensemble_scores.argsort()[::-1]
  # recommended_ensemble = [job for job in pred_user_item_matrix.columns[sorted_ensemble_indices2] if job not in applied_jobs][:10]

  combined_recommendations = recommended_jobs1 + recommended_jobs2 + recommended_jobs3#+ recommended_contents

  # 목록에서 중복 항목을 제거하고 빈도를 계산
  recommendation_counter = Counter(combined_recommendations)

  # 빈도수를 기반으로 상위 5개 추천 얻기
  final_recommendations = [item for item, _ in recommendation_counter.most_common(5)]
  final_recommendations = [item for item in recommened_rating if item in final_recommendations]

  for job in final_recommendations:
      pred_recommendation.append([user, job])

0it [00:00, ?it/s]

0.11872199952841311


0it [00:00, ?it/s]

dot_final: job1, job2, contents(rank)

dot_final_rank: job1(rank), job2(rank), contents(rank) baseline=[:10]
- 0.1909926904032068

dot_final_rank [:15]
- 0.23697241216694176

dot_final_rank [:20]
- 0.27481726008016977

dot_final_rank [:20] _shf
- 0.2680971468993162

dot_final_job_rank: job1(rank), job2(rank)

dot_final_job: job1, job2

dot_final_15samples: dot_final [:15]

In [ ]:
top_recommendations = pd.DataFrame(pred_recommendation, columns=['resume_seq', 'recruitment_seq'])
top_recommendations.to_csv('./dot_final_svd_include.csv', index=False)

In [ ]:
top_recommendations

,resume_seq,recruitment_seq
0,U00001,R02888
1,U00001,R01528
2,U00001,R03811
3,U00001,R03777
4,U00001,R04769
...,...,...
42405,U08482,R02524
42406,U08482,R00473
42407,U08482,R00712
42408,U08482,R05461


In [ ]:
# pred = apply_train.copy().drop('rating_total', axis = 1)
# pred

In [ ]:
#pred['count'] = pred.groupby('resume_seq').cumcount()
#pred

In [ ]:
# # 라벨 인코딩 (필수 아님)
# categorical_cols = []
# for col in notnull_resume.columns:
#   if notnull_resume[col].dtype == 'object':
#     categorical_cols.append(col)

# encoder = LabelEncoder()
# for col in categorical_cols:
#   notnull_resume[col] = encoder.fit_transform(notnull_resume[col])

# categorical_cols = []
# for col in notnull_recruitment.columns:
#   if notnull_recruitment[col].dtype == 'object':
#     categorical_cols.append(col)

# encoder = LabelEncoder()
# for col in categorical_cols:
#   notnull_recruitment[col] = encoder.fit_transform(notnull_recruitment[col])

In [ ]:
# resume_ = resume[['degree', 'graduate_date', 'hope_salary', 'last_salary', 'career_month', 'hischool_location_seq', 'univ_type_seq1', 'univ_type_seq2', 'univ_transfer', 'univ_location', 'univ_major_type', 'univ_score', 'language', 'exam_name', 'score']]
# recruitment_ = recruitment[['address_seq1', 'address_seq2', 'address_seq3', 'career_end', 'career_start', 'education', 'major_task', 'qualifications', 'company_type_seq', 'supply_kind', 'employee']]

In [ ]:
# resume_ = resume_.fillna(0)
# recruitment_ = recruitment_.fillna(0)

In [ ]:
# match_scores = np.zeros((resume_.shape[0],recruitment_.shape[0]))
# for i in resume_.columns:
#   for j in recruitment_.columns:
#     score = cosine_similarity(resume_[i].values.reshape(-1, 1), recruitment_[j].values.reshape(-1, 1))
#     match_scores += score

In [ ]:
# resume_user_list = resume['resume_seq'].unique()
# recommended = []

# for resume_seq in resume_user_list:
#     # 해당 사용자가 지원한 채용 공고
#     applied_jobs = set(user_item_matrix.loc[resume_seq][user_item_matrix.loc[resume_seq] == 1].index)

#     resume_index = resume[resume.resume_seq == resume_seq].index[0]

#     match_scores_recruitment = list(enumerate(match_scores[resume_index]))
#     match_scores_recruitment = sorted(match_scores_recruitment, key=lambda x: x[1], reverse=True)

#     top_N = 5  # 상위 5개 추천된 채용 공고
#     recommended_seqs = [recruitment['recruitment_seq'][recommend[0]] for recommend in match_scores_recruitment[1: top_N*2+1]]

#     # 이미 지원한 채용 공고를 제외하고 상위 N개를 추천
#     filtered_recommended = [seq for seq in recommended_seqs if seq not in applied_jobs][:top_N]

#     for recommended_seq in filtered_recommended:
#         recommended.append([resume_seq, recommended_seq])

In [ ]:
# recommended_predictions = pd.DataFrame(recommended, columns=['resume_seq', 'recruitment_seq'])
# recommended_predictions = recommended_predictions.sort_values(by = 'resume_seq', ascending=True)
# recommended_predictions.to_csv('baseline.csv', index = False)

# 하이브리드 추천 시스템